In [1]:
from datetime import datetime, time, timedelta
from sqlalchemy import create_engine
import pandas as pd
from pandasql import sqldf

# Import Data

In [2]:
cnx = create_engine('postgresql://@localhost/flights')

In [3]:
largest_airports = "('ATL', 'LAX', 'ORD', 'DFW', 'DEN', 'JFK', 'SFO', 'SEA', 'LAS', 'MCO')"

In [4]:
weather_col = 'type, severity, starttimeUTC AS startUTC, endtimeUTC AS endUTC, airport_clean AS airport, city, state'

weather_query = '''SELECT {} 
FROM weather'''.format(weather_col)

weather = (pd.read_sql_query(weather_query, cnx)).astype(
    {'type': 'category', 'severity': 'category', 'airport': 'category', 'city': 'category', 'state': 'category'}
)

weather.head()

type severity            startutc              endutc airport     city  \
0  Snow    Light 2019-12-30 16:55:00 2019-12-30 17:35:00     BNW    Boone   
1  Rain    Light 2019-11-29 23:35:00 2019-11-30 00:15:00     IKV   Ankeny   
2   Fog   Severe 2019-11-29 07:55:00 2019-11-29 08:15:00     AQR    Atoka   
3  Rain    Light 2019-12-22 10:56:00 2019-12-22 11:56:00     AUO  Opelika   
4  Rain    Light 2019-12-28 12:32:00 2019-12-28 12:53:00     TOI     Troy   

  state  
0    IA  
1    IA  
2    OK  
3    AL  
4    AL

In [5]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5059833 entries, 0 to 5059832
Data columns (total 7 columns):
 #   Column    Dtype         
---  ------    -----         
 0   type      category      
 1   severity  category      
 2   startutc  datetime64[ns]
 3   endutc    datetime64[ns]
 4   airport   category      
 5   city      category      
 6   state     category      
dtypes: category(5), datetime64[ns](2)
memory usage: 111.1 MB


In [6]:
timezone_query = '''SELECT timezone, airport_clean AS airport 
FROM weather'''

timezones = (pd.read_sql_query(timezone_query, cnx)).astype('category').drop_duplicates()

In [7]:
timezones['timezone'].unique()

['US/Central', 'US/Mountain', 'US/Eastern', 'US/Pacific']
Categories (4, object): ['US/Central', 'US/Mountain', 'US/Eastern', 'US/Pacific']

In [8]:
flight_col = 'wd.name AS day_of_week, fd.flight_date, fd.reporting_airline, a.name, fd.tail_number, fd.origin, fd.origin_city, fd.origin_statecode, fd.dest, fd.dest_city, fd.dest_statecode, fd.set_dep_time, fd.actual_dep_time, fd.dep_delay, fd.set_arr_time, fd.actual_arr_time, fd.arr_delay, fd.canceled, fd.diverted, fd.set_elapsed_time, fd.actual_elapsed_time, fd.num_of_flights, fd.miles, fd.diverted_landings'

flight_query = '''SELECT {} 
FROM flight_details AS fd 
LEFT OUTER JOIN airlines AS a 
ON fd.reporting_airline = a.ID
LEFT OUTER JOIN weekdays as wd
ON fd.day_of_week = wd.code
WHERE fd.dest IN {}'''.format(flight_col, largest_airports)

flights = (pd.read_sql_query(flight_query, cnx)).astype(
    {'day_of_week': 'category',
     'reporting_airline': 'category',
     'name': 'category',
     'set_dep_time': 'uint16',
     'set_arr_time': 'uint16',
     'canceled': 'uint8',
     'diverted': 'uint8',
     'num_of_flights': 'uint8',
     'miles': 'uint16',
     'diverted_landings': 'uint8'}
)

flights.head()

day_of_week flight_date reporting_airline              name tail_number  \
0      Monday  2019-04-01                NK  Spirit Air Lines      N605NK   
1      Monday  2019-04-08                NK  Spirit Air Lines      N904NK   
2      Monday  2019-04-15                NK  Spirit Air Lines      N637NK   
3      Monday  2019-04-22                NK  Spirit Air Lines      N619NK   
4      Monday  2019-04-29                NK  Spirit Air Lines      N644NK   

  origin    origin_city origin_statecode dest    dest_city  ... set_arr_time  \
0    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...         1245   
1    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...         1245   
2    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...         1245   
3    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...         1245   
4    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...         1245   

   actual_arr_time arr_delay  canceled  diverted set_elapsed_time  \
0             1305      20.0         0         0            145.0   
1             1233     -12.0         0         0            145.0   
2             1241      -4.0         0         0            145.0   
3             1326      41.0         0         0            145.0   
4             1248       3.0         0         0            145.0   

  actual_elapsed_time  num_of_flights  miles diverted_landings  
0               148.0               1    787                 0  
1               134.0               1    787                 0  
2               146.0               1    787                 0  
3               128.0               1    787                 0  
4               135.0               1    787                 0  

[5 rows x 24 columns]

In [9]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2259076 entries, 0 to 2259075
Data columns (total 24 columns):
 #   Column               Dtype   
---  ------               -----   
 0   day_of_week          category
 1   flight_date          object  
 2   reporting_airline    category
 3   name                 category
 4   tail_number          object  
 5   origin               object  
 6   origin_city          object  
 7   origin_statecode     object  
 8   dest                 object  
 9   dest_city            object  
 10  dest_statecode       object  
 11  set_dep_time         uint16  
 12  actual_dep_time      object  
 13  dep_delay            float64 
 14  set_arr_time         uint16  
 15  actual_arr_time      object  
 16  arr_delay            object  
 17  canceled             uint8   
 18  diverted             uint8   
 19  set_elapsed_time     object  
 20  actual_elapsed_time  object  
 21  num_of_flights       uint8   
 22  miles                uint16  
 23  diverte

# Clean Data

## Datetime Columns

In [10]:
flights = pd.merge(flights, timezones, how = 'left', left_on = 'origin', right_on = 'airport')
flights.rename(columns = {'timezone': 'origin_timezone'}, inplace = True)
flights.drop(columns = ['airport'], inplace = True)

flights = pd.merge(flights, timezones, how = 'left', left_on = 'dest', right_on = 'airport')
flights.rename(columns = {'timezone': 'dest_timezone'}, inplace = True)
flights.drop(columns = ['airport'], inplace = True)

In [11]:
time_variables = ['set_dep_time', 'actual_dep_time','set_arr_time','actual_arr_time']

for col in time_variables:
    flights = flights[flights[col] != '']

In [12]:
flights = flights[flights['origin_timezone'].notna()]
flights = flights[flights['dest_timezone'].notna()]

In [13]:
def convert_time(text):
    '''
    Convert string to timestamp datetime object
    
    Arguments
    ----
    text (string): string containing time in 24 clock format
    
    Returns
    ----
    timestamp (time): time listed in text in datetime object 
    '''

    #mil_time = str(int(float(text)))
    mil_time = str(text)
    length = len(mil_time)
                   
    if length < 3:
        minutes = int(mil_time)
        hours = 0
    if length == 3:
        minutes = int(mil_time[:-2])
        hours = int(mil_time[0])
    if length == 4:
        minutes = int(mil_time[:-2])
        if mil_time[:2] == '24':
            hours = 0
        else:
            hours = int(mil_time[:2])
                   
    timestamp = time(hours,minutes)
    
    return timestamp

In [14]:
def convert_timezone(time, from_timezone, to_timezone = 'UTC'):
    '''
    Convert a time to another US timezone, or UTC (converts to UTC by default)
    
    Arguments
    ----
    time (datetime): original time
    from_timezone (str): original timezone, can be any US timezone or UTC (options include US/Eastern, US/Central, US/Mountain, US/Pacific, and UTC)
    to_timezone (str): new timezone, can be any US timezone or UTC (options include US/Eastern, US/Central, US/Mountain, US/Pacific, and UTC)
    
    Returns
    ----
    timestamp (time): time listed in text in datetime object 
    '''
    timezone_list = {'US/Eastern': -5, 'US/Central': -6, 'US/Mountain': -7, 'US/Pacific': -8, 'UTC': 0}
    diff = timezone_list[from_timezone] - timezone_list[to_timezone]
    result = time - timedelta(hours=diff)
    return result

In [16]:
flights['arrtime'] = flights['actual_arr_time'].apply(lambda x: convert_time(x))
flights['deptime'] = flights['actual_dep_time'].apply(lambda x: convert_time(x))
flights['set_arrtime'] = flights['set_arr_time'].apply(lambda x: convert_time(x))
flights['set_deptime'] = flights['set_dep_time'].apply(lambda x: convert_time(x))


flights['dep'] = pd.to_datetime(
    flights['flight_date'].apply(str) + ' ' + flights['deptime'].apply(str)
)
flights['dep_utc'] = flights.apply(lambda x: convert_timezone(x['dep'], x['origin_timezone']), axis=1)
flights['arr_utc'] = flights.apply(
    lambda x: x['dep_utc'] + timedelta(
        minutes = float(x['set_elapsed_time'])
    ), axis = 1
)


flights.drop(columns = time_variables, inplace = True)

In [19]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2154452 entries, 0 to 2259075
Data columns (total 29 columns):
 #   Column               Dtype         
---  ------               -----         
 0   day_of_week          category      
 1   flight_date          object        
 2   reporting_airline    category      
 3   name                 category      
 4   tail_number          object        
 5   origin               object        
 6   origin_city          object        
 7   origin_statecode     object        
 8   dest                 object        
 9   dest_city            object        
 10  dest_statecode       object        
 11  dep_delay            float64       
 12  arr_delay            object        
 13  canceled             uint8         
 14  diverted             uint8         
 15  set_elapsed_time     object        
 16  actual_elapsed_time  object        
 17  num_of_flights       uint8         
 18  miles                uint16        
 19  diverted_landings    

In [ ]:
# timezones.to_csv('timezones.csv')
# weather.to_csv('weather.csv')
# flights.to_csv('flights.csv')

# Merge Datasets

In [3]:
pysqldf = lambda q: sqldf(q, globals())

In [15]:
flights2 = pysqldf(
    '''
    SELECT * 
    FROM flights AS f
    LEFT JOIN weather AS w 
    ON ((f.dep_utc BETWEEN w.startutc AND w.endutc) AND (f.origin = w.airport)) 
    '''
)

flights2.rename(
    columns = {
        'type': 'origin_weather_type', 'severity': 'origin_weather_severity'
    },
    inplace = True
)

flights2.drop(columns = ['startutc', 'endutc', 'airport', 'city', 'state'], inplace = True)

flights2.to_csv('flights2.csv')

In [16]:
flights2.head()

day_of_week flight_date reporting_airline              name tail_number  \
0      Monday  2019-04-01                NK  Spirit Air Lines      N605NK   
1      Monday  2019-04-08                NK  Spirit Air Lines      N904NK   
2      Monday  2019-04-15                NK  Spirit Air Lines      N637NK   
3      Monday  2019-04-22                NK  Spirit Air Lines      N619NK   
4      Monday  2019-04-29                NK  Spirit Air Lines      N644NK   

  origin    origin_city origin_statecode dest    dest_city  ... dest_timezone  \
0    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...    US/Eastern   
1    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...    US/Eastern   
2    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...    US/Eastern   
3    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...    US/Eastern   
4    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...    US/Eastern   

    arrtime   deptime  set_arrtime  set_deptime                  dep  \
0  13:13:00  10:10:00     12:12:00     10:10:00  2019-04-01 10:10:00   
1  12:12:00  10:10:00     12:12:00     10:10:00  2019-04-08 10:10:00   
2  12:12:00  10:10:00     12:12:00     10:10:00  2019-04-15 10:10:00   
3  13:13:00  11:11:00     12:12:00     10:10:00  2019-04-22 11:11:00   
4  12:12:00  10:10:00     12:12:00     10:10:00  2019-04-29 10:10:00   

               dep_utc              arr_utc  origin_weather_type  \
0  2019-04-01 15:10:00  2019-04-01 17:35:00                 None   
1  2019-04-08 15:10:00  2019-04-08 17:35:00                 None   
2  2019-04-15 15:10:00  2019-04-15 17:35:00                Storm   
3  2019-04-22 16:11:00  2019-04-22 18:36:00                 None   
4  2019-04-29 15:10:00  2019-04-29 17:35:00                 None   

   origin_weather_severity  
0                     None  
1                     None  
2                   Severe  
3                     None  
4                     None  

[5 rows x 31 columns]

In [17]:
final = pysqldf(
    '''
    SELECT * 
    FROM flights2 AS f
    LEFT JOIN weather AS w 
    ON ((f.arr_utc BETWEEN w.startutc AND w.endutc) AND (f.dest = w.airport)) 
    '''
)

final.rename(
    columns = {
        'type': 'dest_weather_type', 'severity': 'dest_weather_severity'
    },
    inplace = True
)

final.drop(columns = ['startutc', 'endutc', 'airport', 'city', 'state'], inplace = True)

final.to_csv('final.csv')

In [18]:
final.head()

day_of_week flight_date reporting_airline              name tail_number  \
0      Monday  2019-04-01                NK  Spirit Air Lines      N605NK   
1      Monday  2019-04-08                NK  Spirit Air Lines      N904NK   
2      Monday  2019-04-15                NK  Spirit Air Lines      N637NK   
3      Monday  2019-04-22                NK  Spirit Air Lines      N619NK   
4      Monday  2019-04-29                NK  Spirit Air Lines      N644NK   

  origin    origin_city origin_statecode dest    dest_city  ...   deptime  \
0    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...  10:10:00   
1    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...  10:10:00   
2    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...  10:10:00   
3    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...  11:11:00   
4    BWI  Baltimore, MD               MD  MCO  Orlando, FL  ...  10:10:00   

   set_arrtime  set_deptime                  dep              dep_utc  \
0     12:12:00     10:10:00  2019-04-01 10:10:00  2019-04-01 15:10:00   
1     12:12:00     10:10:00  2019-04-08 10:10:00  2019-04-08 15:10:00   
2     12:12:00     10:10:00  2019-04-15 10:10:00  2019-04-15 15:10:00   
3     12:12:00     10:10:00  2019-04-22 11:11:00  2019-04-22 16:11:00   
4     12:12:00     10:10:00  2019-04-29 10:10:00  2019-04-29 15:10:00   

               arr_utc  origin_weather_type  origin_weather_severity  \
0  2019-04-01 17:35:00                 None                     None   
1  2019-04-08 17:35:00                 None                     None   
2  2019-04-15 17:35:00                Storm                   Severe   
3  2019-04-22 18:36:00                 None                     None   
4  2019-04-29 17:35:00                 None                     None   

   dest_weather_type  dest_weather_severity  
0               None                   None  
1               None                   None  
2               None                   None  
3               None                   None  
4               None                   None  

[5 rows x 33 columns]